In [38]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import pickle
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle


In [66]:
# This is the raw text
file = open('../dataset/news_pickles/train_test_val_Human', 'rb')
X_human_train, X_human_test, X_human_val = pickle.load(file)

X_human_train[0]


'Last month a controversial facial recognition company, Clearview AI, announced it had given its technology to the Ukrainian government. The BBC has been given evidence of how it is being used - in more than a thousand cases - to identify both the living and the dead.  This story contains graphic descriptions that may be upsetting to some readers. A man lies motionless on the floor, his head tilted down. His body is naked, apart from a pair of Calvin Klein boxers. His eyes are ringed with what look like bruises. The body was found in Kharkiv, eastern Ukraine - in the wreckage of war. The BBC has seen pictures taken at the scene, but does not know the circumstances around his death. There is clear evidence of head trauma. He also had a tattoo on his left shoulder.  Ukrainian authorities didn\'t know who the man was, so decided to turn to a cutting edge method: facial recognition using artificial intelligence.  Clearview is perhaps the most famous, and controversial, facial recognition s

## Data processing

In [39]:
file = open('../dataset/news_pickles/train_test_val_Human_posTags', 'rb')
X_human_train_posTags, X_human_test_posTags, X_human_val_posTags = pickle.load(file)
file.close()

Y_human_train = np.zeros(len(X_human_train_posTags))
Y_human_test = np.zeros(len(X_human_test_posTags))
Y_human_val = np.zeros(len(X_human_val_posTags))


file = open('../dataset/news_pickles/train_test_val_GPT_posTags', 'rb')
X_gpt_train_posTags, X_gpt_test_posTags, X_gpt_val_posTags = pickle.load(file)
file.close()

Y_gpt_train = np.ones(len(X_gpt_train_posTags))
Y_gpt_test = np.ones(len(X_gpt_test_posTags))
Y_gpt_val = np.ones(len(X_gpt_val_posTags))

In [40]:
len(X_human_train_posTags), len(Y_human_train), len(X_human_test_posTags), len(Y_human_test), len(X_human_val_posTags), len(Y_human_val)

(435, 435, 95, 95, 93, 93)

In [41]:
X_train = np.hstack((X_human_train_posTags, X_gpt_train_posTags))
Y_train = np.hstack((Y_human_train, Y_gpt_train))

X_test = np.hstack((X_human_test_posTags, X_gpt_test_posTags))
Y_test = np.hstack((Y_human_test, Y_gpt_test))

X_val = np.hstack((X_human_val_posTags, X_gpt_val_posTags))
Y_val = np.hstack((Y_human_val, Y_gpt_val))


full_dataset = np.hstack((X_train, X_test, X_val))

In [42]:
X_train = [' '.join(data) for data in X_train]
X_test = [' '.join(data) for data in X_test]
X_val = [' '.join(data) for data in X_val]

full_dataset = [' '.join(data) for data in full_dataset]


In [63]:
vocab_size = 55
oov_tok = ''
embedding_dim = 150
max_length = 200

padding_type='post'
trunc_type='post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

tokenizer.fit_on_texts(full_dataset)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

val_sequences = tokenizer.texts_to_sequences(X_val)
val_padded = pad_sequences(val_sequences, padding='post', maxlen=max_length)

# Assuming 'tokenizer' is your trained Keras Tokenizer. SAVE THIS TO USE IN PREDICTION
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

X_train

["JJ NN DT JJ JJ NN NN , NNP NNP , VBD PRP VBD VBN PRP$ NN TO DT JJ NN . DT NNP VBZ VBN VBN NN IN WRB PRP VBZ VBG VBN : IN JJR IN DT NN NNS : TO VB DT DT NN CC DT JJ . DT NN VBZ JJ NNS WDT MD VB VBG TO DT NNS . DT NN VBZ RBR IN DT NN , PRP$ NN VBD RB . PRP$ NN VBZ JJ , RB IN DT NN IN NNP NNP NNS . PRP$ NNS VBP VBN IN WP VBP JJ NNS . DT NN VBD VBN IN NNP , JJ NNP : IN DT NN IN NN . DT NNP VBZ VBN NNS VBN IN DT NN , CC VBZ RB VB DT NNS IN PRP$ NN . EX VBZ JJ NN IN NN NN . PRP RB VBD DT NN IN PRP$ JJ NN . JJ NNS VBD RB VB WP DT NN VBD , RB VBD TO VB TO DT VBG NN NN : JJ NN VBG JJ NN . NNP VBZ RB DT RBS JJ , CC JJ , JJ NN NN IN DT NN . DT NN VBZ VBN NNS IN NNS IN JJ NNS NNS , IN NNP CC NNP , TO VB DT JJ NN IN WP PRP$ NN CC NN NNP NNP VBZ `` DT NN NN IN VBZ '' . `` PRP NN IN NNS IN NNP . CC RB IN VBG IN DT NN IN NNS CC NN , DT NN VBZ IN DT NN IN DT NN , '' VBZ NNP NNP . DT NN VBZ VBN DT NN IN JJ NNS . NNP , NNP , NNP CC NNP VBP VBN JJ NNS TO NNP : TO VB PRP TO VB VBG NNS IN DT NNS . DT NNP 

In [44]:
train_padded, Y_train = shuffle(train_padded, Y_train)
test_padded, Y_test = shuffle(test_padded, Y_test)
val_padded, Y_val = shuffle(val_padded, Y_val)

In [64]:
train_padded.shape, val_padded.shape
train_padded

array([[ 3, 10, 12, ...,  5,  5,  2],
       [ 3,  7,  3, ...,  5,  5,  2],
       [ 4,  7,  3, ...,  5,  5,  5],
       ...,
       [26,  6,  7, ...,  4,  6,  7],
       [ 7,  3,  6, ...,  7, 14,  2],
       [ 6,  2, 20, ...,  2,  3,  4]], dtype=int32)

## LSTM Model

In [46]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])


model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 150)          8250      
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               110080    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 122491 (478.48 KB)
Trainable params: 122491 (478.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [62]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

callback = EarlyStopping(monitor='accuracy', patience=5)
train_padded

array([[ 3, 10, 12, ...,  5,  5,  2],
       [ 3,  7,  3, ...,  5,  5,  2],
       [ 4,  7,  3, ...,  5,  5,  5],
       ...,
       [26,  6,  7, ...,  4,  6,  7],
       [ 7,  3,  6, ...,  7, 14,  2],
       [ 6,  2, 20, ...,  2,  3,  4]], dtype=int32)

In [48]:
num_epochs = 27
history = model.fit(train_padded, Y_train, 
                    epochs=num_epochs, 
                    verbose=1, 
                    shuffle=True,
                    validation_data=(val_padded, Y_val),
                    callbacks=[callback])

Epoch 1/27
28/28 [==============================] - 10s 266ms/step - loss: 0.6083 - accuracy: 0.6816 - val_loss: 0.4387 - val_accuracy: 0.7742
Epoch 2/27
28/28 [==============================] - 8s 270ms/step - loss: 0.3019 - accuracy: 0.8851 - val_loss: 0.4233 - val_accuracy: 0.7688
Epoch 3/27
28/28 [==============================] - 7s 263ms/step - loss: 0.2927 - accuracy: 0.8816 - val_loss: 0.3216 - val_accuracy: 0.8656
Epoch 4/27
28/28 [==============================] - 10s 346ms/step - loss: 0.2311 - accuracy: 0.9057 - val_loss: 0.2614 - val_accuracy: 0.8817
Epoch 5/27
28/28 [==============================] - 11s 376ms/step - loss: 0.2300 - accuracy: 0.9230 - val_loss: 0.2594 - val_accuracy: 0.9032
Epoch 6/27
28/28 [==============================] - 11s 376ms/step - loss: 0.2449 - accuracy: 0.9069 - val_loss: 0.4075 - val_accuracy: 0.8333
Epoch 7/27
28/28 [==============================] - 11s 377ms/step - loss: 0.1947 - accuracy: 0.9230 - val_loss: 0.2281 - val_accuracy: 0.9247
E

In [52]:
model.save('../classifiers/trained_models/LSTM_high_acc9437_9140_9157.h5')

In [53]:
score = model.evaluate(test_padded, Y_test, verbose=2)
print('Evaluation loss:', score[0])
print('Evaluation accuracy:', score[1])

6/6 - 0s - loss: 0.3309 - accuracy: 0.9053 - 324ms/epoch - 54ms/step
Evaluation loss: 0.33089977502822876
Evaluation accuracy: 0.9052631855010986
